In [4]:
import sqlite3
import pandas as pd

### sqlite3 db to Dataframe

In [5]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [6]:
df_store = pd.read_sql_query("SELECT * from api_store", con)
df_review = pd.read_sql_query("SELECT * from api_storereview", con)

In [7]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [8]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-07 05:48:37.271693,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-07 05:48:37.271693,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-07 05:48:37.271693,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-07 05:48:37.271693,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-07 05:48:37.271693,38,115682


In [9]:
con.close()

### surprise
#### https://medium.com/sfu-cspmp/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0
#### https://github.com/ashaypathak/Recommendation-system/blob/master/Movie_Recommendation.ipynb

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [20]:
stores = df_store.loc[:,['id','store_name','category','address']]
reviews = df_review.loc[:,['user_id','store_id','total_score']]
reviews.rename(columns={'total_score':'rating'}, inplace=True)
stores.rename(columns={'id':'store_id'},inplace=True)
df =pd.merge(stores, reviews,on="store_id")
df.head()

,store_id,store_name,category,address,user_id,rating
0,15,써리힐,호주레스토랑,부산광역시 부산진구 전포동 229-13번지 2층,68632,5.0
1,18,진삼미 샌드위치,샌드위치|쥬스,전라북도 전주시 완산구 효자동1가 652 상산고등학교앞,389728,5.0
2,19,한옥마을 전주비빔밥,전주비빔밥,경기도 화성시 향남읍 행정리 487-1 홈플러스 화성향남점 1층,68716,4.0
3,37,007식당,굴비정식|굴비,전라남도 영광군 법성면 법성리 1148-6,774353,2.0
4,38,010수산,횟집,서울특별시 강서구 등촌동 671-3,115682,3.0


https://www.fun-coding.org/recommend_basic7.html

#### https://lsjsj92.tistory.com/570?category=853217

#### https://antilibrary.org/2086

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import dump

In [17]:
#swapping columns
raw=df[['user_id','store_id','rating']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = Dataset.load_from_df(raw,reader)

NameError: name 'df' is not defined

In [12]:
# 트레인 데이터로 학습
algo = surprise.SVD()
# cross_validate(algo, dataframe,measures=['RMSE'], cv=3, verbose=True)
algo.fit(dataframe.build_full_trainset())

#### 서울 지역의 유저아이디 4974에 대한 추천 가게 리스트 생성 

In [13]:
test = df[df['address'].str.contains("서울")]

In [14]:
user_id = 4974

In [15]:
iids = test['store_id'].unique()
iidsUsrnotVisited = raw.loc[raw['user_id']==user_id, 'store_id']
iids_to_pred = np.setdiff1d(iids,iidsUsrnotVisited) # 안 간 가게 구함(차집합)

In [16]:
testset = [[user_id, iid, 5.] for iid in iids_to_pred]
predictions = algo.test(testset)

Prediction(uid=4974, iid=38, r_ui=5.0, est=3.721401705082229, details={'was_impossible': False})

In [30]:
pred_ratings = np.array([pred.est for pred in predictions])
print(pred_ratings)

[3.72140171 3.55358597 3.72654784 ... 3.4389243  3.77566638 3.44317783]


In [21]:
i_max = pred_ratings.argsort()[::-1][:10]
#i_max = pred_ratings.argmax()
iid = iids_to_pred[i_max]
print('Top store for user has store_id {0} with predicted rating {1}'.format(iid,pred_ratings[i_max]))

Top store for user has store_id [152068  74520 118035    221 129842  90781 338854 157481  95034  70803] with predicted rating [4.59753931 4.48574456 4.41801832 4.40197092 4.38460441 4.38008029
 4.36248214 4.3529043  4.35155343 4.33595085]


In [22]:
for i,m in zip(iid,i_max):
    print()
    print('{0} : {1}'.format(i,pred_ratings[m]))


152068 : 4.597539308164149

74520 : 4.485744558815989

118035 : 4.418018316306834

221 : 4.401970924430185

129842 : 4.384604414647126

90781 : 4.380080288870979

338854 : 4.362482141247682

157481 : 4.352904299233391

95034 : 4.351553432615645

70803 : 4.335950851324036


##### 정확도 계산
from surprise import accuracy
 ##### 전체 데이터로 testset 해서 측정
testset = trainset.build_testset()
predictions = algo.test(testset)

accuracy.rmse(predictions)

def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [62]:
import os
import surprise
# 예측모델 지정해주기 ~~!
def set_algo(user_id):
    # dump file 지정
    file_name = os.path.expanduser('dump_file')

    # load dataframe
    #swapping columns
    raw=df[['user_id','store_id','rating']] 
    # when importing from a DF, you only need to specify the scale of the ratings.
    reader = surprise.Reader(rating_scale=(0,5)) 
    #into surprise:
    dataframe = surprise.Dataset.load_from_df(raw,reader)
    trainset = dataframe.build_full_trainset()

    try:
        _, algo = surprise.dump.load(file_name)
     
    except FileNotFoundError:
        # 파일이 존재하지않는 경우
        print("Dump file Doesn't exists.")
        # Use the famous SVD algorithm.
        algo = surprise.SVD()
        algo.fit(trainset)
        surprise.dump.dump(file_name, algo=algo)

    finally:
        test = df[df['address'].str.contains("사동")]
        iids = test['store_id'].unique()
        iidsUsrnotVisited = raw.loc[raw['user_id']==user_id, 'store_id']
        iids_to_pred = np.setdiff1d(iids,iidsUsrnotVisited) # 안 간 가게 구함(차집합)
        # user_id가 가지않은 가게들로 testset 생성
        testset = [[user_id, iid, 5.] for iid in iids_to_pred]
        predictions = algo.test(testset)
        print(surprise.accuracy.rmse(predictions))
        pred_ratings = np.array([pred.est for pred in predictions])
        i_max = pred_ratings.argsort()[::-1][:10] # 역순으로 상위 10개
        #i_max = pred_ratings.argmax()
        iid = iids_to_pred[i_max]
        results = []
        for i,m in zip(iid,i_max):
            print('{0} : {1}'.format(i,pred_ratings[m]))
            results.append({i:pred_ratings[m]})
            
        
        return results,iid

In [63]:
result,iid = set_algo(4974)

RMSE: 1.4490
1.449014832446019
96612 : 4.101700973352529
166053 : 4.052025022992504
76201 : 4.032114930503674
131833 : 4.0158733652172485
262120 : 4.010808294747416
47153 : 3.98644983320177
125236 : 3.9816749149147146
142481 : 3.9796883645433714
266711 : 3.9792510571041024
2334 : 3.9777919654723424


In [64]:
t= pd.DataFrame(iid,columns=['store_id'])

In [65]:
iid

array([ 96612, 166053,  76201, 131833, 262120,  47153, 125236, 142481,
       266711,   2334], dtype=int64)

In [66]:
pd.merge(stores,t,on="store_id",how='inner')
# 추천 결과

,store_id,store_name,category,address
0,2334,3촌,삼겹살|고기집,울산광역시 중구 약사동 591-5
1,47153,금수복국,복국|은복,서울특별시 강남구 신사동 627-13
2,76201,다운타우너,아보카도버거|수제버거,서울특별시 강남구 신사동 666-25
3,96612,도쿄등심,오마카세|한우,서울특별시 강남구 신사동 651-19 스타크빌딩 3층
4,125236,리스토란테33,파스타|레스토랑,경상북도 경산시 사동 33
5,131833,만,장어|장어요리,서울특별시 강남구 신사동 636-23
6,142481,메이첸팡,중국집|짜장면,경기도 고양시 일산동구 식사동 1537-1
7,166053,바빌리안테이블,레스토랑|치즈퐁듀,서울특별시 강남구 신사동 660-18
8,262120,아우어베이커리,베이커리|카페,서울특별시 강남구 신사동 523-19
9,266711,압구정면옥,냉면,서울특별시 강남구 신사동 610-3


### 위도 경도로 거리차이 구하기

In [231]:
import math
import numbers
def get_euclidean_distance(x1,y1,x2,y2,round_decimal_digits=5):
    if x1 is None or y1 is None or x2 is None or y2 is None:
        return None
    assert isinstance(x1, numbers.Number) and -180 <=x1 and x1<=180
    assert isinstance(y1, numbers.Number) and -90 <=y1 and y1<=90
    assert isinstance(x2, numbers.Number) and -180 <=x2 and x2<=180
    assert isinstance(y2, numbers.Number) and -90 <=y2 and y2<=90
    dLon = abs(x2-x1) # 경도차이
    if dLon >=180:
        dLon -=360
    dLat = y2-y1 # 위도차이
    return round(math.sqrt(pow(dLon,2) + pow(dLat,2)), round_decimal_digits)

a = get_euclidean_distance(-179,0,179,0)

### 참고 소스인데 않뎀..

In [10]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import dump
import surprise
import os

#### Memory error ... build_anti_testset()에서 나는듯..

In [ ]:
#swapping columns
df = pd.merge(reviews, stores,on="store_id")
raw=df[['user_id','store_id','rating']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)
trainset = dataframe.build_full_trainset()
# Use the famous SVD algorithm.
algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()

predictions = algo.test(trainset.build_anti_testset())

### algofile dump 만드리

In [ ]:
# Dump algorithm and reload it.
file_name = os.path.expanduser('dump_file')
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name)

# We now ensure that the algo is still the same by checking the predictions.
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
assert predictions == predictions_loaded_algo
print('Predictions are the same')

In [ ]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

# 성능검사
algo = surprise.SVD()
cross_validate(algo, dataframe,measures=['RMSE','MAE'], cv=5, verbose=True)